# Dask Arrays

<img src="images/dask-array-black-text.svg" 
     align="right"
     alt="Dask arrays are blocked numpy arrays">
     
Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid.  They support a large subset of the Numpy API.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.251/12030/1 Dashboard: http://10.20.0.251:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create Random array

This creates a 10000x10000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). In this case there are 100 (10x10) numpy arrays of size 1000x1000.

In [2]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

Use NumPy syntax as usual

In [3]:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

dask.array<mean_agg-aggregate, shape=(5000,), dtype=float64, chunksize=(500,), chunktype=numpy.ndarray>

Call `.compute()` when you want your result as a NumPy array.

If you started `Client()` above then you may want to watch the status page during computation.

In [4]:
z.compute()

array([1.00168976, 0.99825704, 0.99052903, ..., 0.99582598, 1.00283016,
       0.99481762])

## Persist data in memory

If you have the available RAM for your dataset then you can persist data in memory.  

This allows future computations to be much faster.

In [5]:
y = y.persist()

In [6]:
%time y[0, 0].compute()

CPU times: user 2.2 s, sys: 216 ms, total: 2.42 s
Wall time: 1.24 s


0.14051977499889556

In [7]:
%time y.sum().compute()

CPU times: user 322 ms, sys: 9.62 ms, total: 331 ms
Wall time: 235 ms


100009122.17207699

## Further Reading 

A more in-depth guide to working with Dask arrays can be found in the [dask tutorial](https://github.com/dask/dask-tutorial), notebook 03.